In [1]:
#Librerias
import pandas as pd


In [2]:
# ————————————————————————————————————————————————
# 1) Propiedades físico-químicas obtenidas de la literatura
# Cada material incluye su conductividad electrónica, difusividad iónica
# y permitividad relativa (adimensional), extraídas de fuentes académicas.
# ————————————————————————————————————————————————

propiedades_lit = {
    "Al": {
        # Aluminio comercial: σ ≃ 3.5×10⁷ S/m
        "sigma_electronica": 3.5e7,
        "difusion_ionica": 0.0,
        "permitividad_relativa": 1.0
    },
    "Al|NMC": {
        # Al2O3 pasivada: σ ≃ 1×10⁻² S/m
        "sigma_electronica": 1e-2,
        "difusion_ionica": 1e-15,
        "permitividad_relativa": 15.0
    },
    "NMC811 (con grafeno)": {
        # NMC811 con aditivo conductor: σ ≃ 2.1 S/m
        "sigma_electronica": 2.1,
        "difusion_ionica": 1e-14,
        "permitividad_relativa": 20.0
    },
    "NMC|LLZO": {
        # Interfase de LiNbO3: σ_ion ≃ 1e-4 S/m
        "sigma_electronica": 1e-8,
        "difusion_ionica": 1e-13,
        "permitividad_relativa": 30.0
    },
    "LLZO": {
        # Electrolito LLZO puro: σ_ion ≃ 2.9e-3 S/m
        "sigma_electronica": 1e-10,
        "difusion_ionica": 1e-13,
        "permitividad_relativa": 60.0
    },
    "LLZO|Li": {
        # Capa pasiva de Li2CO3: σ_elec ≃ 1e-14 S/m
        "sigma_electronica": 1e-14,
        "difusion_ionica": 1e-14,
        "permitividad_relativa": 10.0
    },
    "Li": {
        # Litio metálico: σ ≃ 1.1×10⁷ S/m
        "sigma_electronica": 1.1e7,
        "difusion_ionica": 1e-10,
        "permitividad_relativa": 1.1
    }
}

# ————————————————————————————————————————————————
# 2) Densidades (kg/m³) de los materiales
# ————————————————————————————————————————————————

densidades = {
    "Al": 2700,
    "NMC811 (con grafeno)": 4500,
    "LLZO": 5100,
    "Li": 5300,
    "Al|NMC": 3980,       # Al2O3
    "NMC|LLZO": 4647,     # LiNbO3
    "LLZO|Li": 2110       # Li2CO3
}

# ————————————————————————————————————————————————
# 3) Masas molares (kg/mol) de cada componente
# ————————————————————————————————————————————————

masas_molares = {
    "Al": 0.02698,
    "NMC811 (con grafeno)": 0.0978,
    "LLZO": 0.8397,
    "Li": 0.00694,
    "Al|NMC": 0.10196,
    "NMC|LLZO": 0.14785,
    "LLZO|Li": 0.07389
}

# ————————————————————————————————————————————————
# 4) Espesores nominales (en metros) de cada capa
# ————————————————————————————————————————————————

espesores_base = {
    "Al": 2.0e-5,
    "NMC811 (con grafeno)": 7.5e-5,
    "LLZO": 2.0e-4,
    "Li": 7.0e-5
}

espesores_interfaces = {
    "Al|NMC": 5e-8,
    "NMC|LLZO": 1e-7,
    "LLZO|Li": 1e-7
}

# Cálculo del espesor efectivo (material base menos mitad de cada interfaz adyacente)
espesores_efectivos = {}
for mat, e0 in espesores_base.items():
    adj = [i for i in espesores_interfaces if mat in i.split("|")]
    descuento = sum(espesores_interfaces[i] for i in adj) * 0.5
    espesores_efectivos[mat] = e0 - descuento

# Unimos todos los espesores efectivos con las capas de interfaz completas
espesores_total = {**espesores_efectivos, **espesores_interfaces}

# ————————————————————————————————————————————————
# 5) Definimos el orden físico de las capas
# ————————————————————————————————————————————————

orden_capas = [
    "Al",
    "Al|NMC",
    "NMC811 (con grafeno)",
    "NMC|LLZO",
    "LLZO",
    "LLZO|Li",
    "Li"
]

espesores = {mat: espesores_total[mat] for mat in orden_capas}

# ————————————————————————————————————————————————
# 6) Definición de iones de litio por unidad fórmula
# y fracción móvil en cada material
# ————————————————————————————————————————————————

iones_por_formula = {
    "Al": 0,
    "NMC811 (con grafeno)": 1,
    "LLZO": 7,
    "Li": 1,
    "Al|NMC": 0,
    "NMC|LLZO": 1,
    "LLZO|Li": 2
}

fraccion_movil = {
    m: 0.3 if m == "LLZO" else 1.0
    for m in espesores_total
}

# ————————————————————————————————————————————————
# 7) Cálculo de propiedades por unidad de superficie
# ————————————————————————————————————————————————

masas_por_m2 = {
    m: densidades[m] * espesores[m]
    for m in orden_capas
}

moles_por_m2 = {
    m: masas_por_m2[m] / masas_molares[m]
    for m in orden_capas
}

moles_Li_efectivos = {
    m: moles_por_m2[m] * iones_por_formula[m] * fraccion_movil[m]
    for m in orden_capas
}

# ————————————————————————————————————————————————
# 8) Creación del DataFrame resumen
# ————————————————————————————————————————————————

df = pd.DataFrame({
    "Material": orden_capas,
    "Espesor (m)": [espesores[m] for m in orden_capas],
    "Densidad (kg/m³)": [densidades[m] for m in orden_capas],
    "Masa/m² (kg/m²)": [masas_por_m2[m] for m in orden_capas],
    "Moles/m²": [moles_por_m2[m] for m in orden_capas],
    "Li⁺ efectivo/m²": [moles_Li_efectivos[m] for m in orden_capas],
    "σ [S/m]": [propiedades_lit[m]["sigma_electronica"] for m in orden_capas],
    "D iónica [m²/s]": [propiedades_lit[m]["difusion_ionica"] for m in orden_capas],
    "ε_r": [propiedades_lit[m]["permitividad_relativa"] for m in orden_capas]
})

# Mostrar resultados
print("Resumen de espesores (ordenados):\n")
for mat, esp in espesores.items():
    print(f"{mat:20} → {esp:.2e} m")

print("\nResumen general de propiedades por m²:\n")
print(df.to_string(index=False))


Resumen de espesores (ordenados):

Al                   → 2.00e-05 m
Al|NMC               → 5.00e-08 m
NMC811 (con grafeno) → 7.50e-05 m
NMC|LLZO             → 1.00e-07 m
LLZO                 → 2.00e-04 m
LLZO|Li              → 1.00e-07 m
Li                   → 6.99e-05 m

Resumen general de propiedades por m²:

            Material  Espesor (m)  Densidad (kg/m³)  Masa/m² (kg/m²)  Moles/m²  Li⁺ efectivo/m²      σ [S/m]  D iónica [m²/s]  ε_r
                  Al 1.997500e-05              2700         0.053933  1.998981         0.000000 3.500000e+07     0.000000e+00  1.0
              Al|NMC 5.000000e-08              3980         0.000199  0.001952         0.000000 1.000000e-02     1.000000e-15 15.0
NMC811 (con grafeno) 7.500000e-05              4500         0.337500  3.450920         3.450920 2.100000e+00     1.000000e-14 20.0
            NMC|LLZO 1.000000e-07              4647         0.000465  0.003143         0.003143 1.000000e-08     1.000000e-13 30.0
                LLZO 1.999000e-